In [1]:
import pandas as pd

In [2]:
df_eleitorados = pd.read_csv("eleitorado/perfil_eleitorado_2020/perfil_eleitorado_2020.csv", encoding="Latin 1", quotechar='"', sep=";"  )

In [3]:
df_resultados = pd.read_csv("test.csv", encoding="Latin 1", on_bad_lines='skip', quotechar='"', sep=";")

In [4]:
col_nao_necessarias_eleitorados = ["DT_GERACAO", "HH_GERACAO", "ANO_ELEICAO", "CD_MUN_SIT_BIOMETRIA", "DS_MUN_SIT_BIOMETRIA", "QT_ELEITORES_BIOMETRIA", "QT_ELEITORES_DEFICIENCIA", "QT_ELEITORES_INC_NM_SOCIAL", "SG_UF"]
col_nao_necessarias_resultados = ["DT_GERACAO", "HH_GERACAO", "ANO_ELEICAO", "CD_TIPO_ELEICAO", "NM_TIPO_ELEICAO", "DT_PLEITO", "NR_TURNO", "DS_ELEICAO", "SG_UF", "DS_CARGO_PERGUNTA", "DT_BU_RECEBIDO", "QT_ABSTENCOES", "CD_TIPO_URNA", "DS_TIPO_URNA", "NR_URNA_EFETIVADA", "CD_CARGA_1_URNA_EFETIVADA", "CD_CARGA_2_URNA_EFETIVADA", "CD_FLASHCARD_URNA_EFETIVADA", "DT_CARGA_URNA_EFETIVADA", "DS_AGREGADAS", "DT_ABERTURA", "DT_ENCERRAMENTO", "QT_ELEITORES_BIOMETRIA_NH", "NR_JUNTA_APURADORA", "NR_TURMA_APURADORA"]


In [5]:
for col in col_nao_necessarias_eleitorados:
    df_eleitorados = df_eleitorados.drop(col, axis=1)

In [6]:
for col_ in col_nao_necessarias_resultados:
    df_resultados = df_resultados.drop(col_, axis=1)